In [1]:
import fitz  # PyMuPDF
from langchain_core.documents import Document
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch 
import numpy as np
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage
from sklearn.metrics.pairwise import cosine_similarity
import os
import base64
import io
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import ollama_python as ollama
import requests
import json

/media/anishbawdhankar/Study/Study/Desktop/RAG_v1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
###Clip Model
import os
from dotenv import load_dotenv
load_dotenv()

### initialize the Clip Model for unified embeddings
clip_model=CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor=CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [3]:
### Embedding functions
def embed_image(image_data):
    """Embed image using CLIP"""
    if isinstance(image_data, str):  # If path
        image = Image.open(image_data).convert("RGB")
    else:  # If PIL Image
        image = image_data
    
    inputs=clip_processor(images=image,return_tensors="pt")
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        # Normalize embeddings to unit vector
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()
    
def embed_text(text):
    """Embed text using CLIP."""
    inputs = clip_processor(
        text=text, 
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=77  # CLIP's max token length
    )
    with torch.no_grad():
        features = clip_model.get_text_features(**inputs)
        # Normalize embeddings
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

In [4]:
## Process PDF
pdf_path="test_onlytext.pdf"
doc=fitz.open(pdf_path)
# Storage for all documents and embeddings
all_docs = []
all_embeddings = []
image_data_store = {}  # Store actual image data for LLM

# Text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)




In [5]:
doc

Document('test_onlytext.pdf')

In [6]:
for i,page in enumerate(doc):
    ## process text
    text=page.get_text()
    if text.strip():
        ##create temporary document for splitting
        temp_doc = Document(page_content=text, metadata={"page": i, "type": "text"})
        text_chunks = splitter.split_documents([temp_doc])

        #Embed each chunk using CLIP
        for chunk in text_chunks:
            embedding = embed_text(chunk.page_content)
            all_embeddings.append(embedding)
            all_docs.append(chunk)



    ## process images
    ##Three Important Actions:

    ##Convert PDF image to PIL format
    ##Store as base64 for GPT-4V (which needs base64 images)
    ##Create CLIP embedding for retrieval

    for img_index, img in enumerate(page.get_images(full=True)):
        try:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            
            # Convert to PIL Image
            pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
            
            # Create unique identifier
            image_id = f"page_{i}_img_{img_index}"
            
            # Store image as base64 for later use with GPT-4V
            buffered = io.BytesIO()
            pil_image.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode()
            image_data_store[image_id] = img_base64
            
            # Embed image using CLIP
            embedding = embed_image(pil_image)
            all_embeddings.append(embedding)
            
            # Create document for image
            image_doc = Document(
                page_content=f"[Image: {image_id}]",
                metadata={"page": i, "type": "image", "image_id": image_id}
            )
            all_docs.append(image_doc)
            
        except Exception as e:
            print(f"Error processing image {img_index} on page {i}: {e}")
            continue

doc.close()


In [7]:
all_docs

[Document(metadata={'page': 0, 'type': 'text'}, page_content="Delhi team \nKohli's junior cricket career kicked off in October 2002 at the Luhnu Cricket Ground \nagainst Himachal Pradesh. His first half-century in domestic cricket happened at \nFeroze Shah Kotla, where he scored 70 runs against Haryana.[12] By the end of the \nseason, he had amassed a total of 172 runs, emerging as the highest run-scorer for \nhis side with an average of 34.40.[13] During the 2003–04 season of Polly Umrigar"),
 Document(metadata={'page': 0, 'type': 'text'}, page_content="his side with an average of 34.40.[13] During the 2003–04 season of Polly Umrigar \nTrophy, Kohli was appointed the captain of the U-15 team.[13] He scored 54 runs in \nDelhi's victory over Himachal Pradesh. In the next fixture against Jammu and \nKashmir, Kohli scored his maiden century with a score of 119. By the end of the \nseason, he had a total of 390 runs at an average of 78, which included two \ncenturies.[14] Towards the end o

In [8]:
# Create unified FAISS vector store with CLIP embeddings
embeddings_array = np.array(all_embeddings)
embeddings_array

array([[-0.01221295,  0.02310531, -0.01041862, ...,  0.07908116,
         0.0321408 ,  0.01603506],
       [-0.01322551,  0.00416262, -0.00712366, ...,  0.00750809,
         0.03890461,  0.01258403],
       [-0.01738388,  0.04637602, -0.02278408, ...,  0.03009844,
         0.04086243,  0.03106904],
       ...,
       [ 0.00984037,  0.01156813, -0.00213682, ...,  0.08411917,
         0.01528242,  0.00601851],
       [-0.01412709,  0.0111115 , -0.00147218, ...,  0.02354532,
         0.02447616,  0.02000337],
       [ 0.02731348, -0.0009631 ,  0.0191607 , ...,  0.03305102,
         0.0170649 ,  0.00761075]], shape=(15, 512), dtype=float32)

In [9]:
(all_docs,embeddings_array)

([Document(metadata={'page': 0, 'type': 'text'}, page_content="Delhi team \nKohli's junior cricket career kicked off in October 2002 at the Luhnu Cricket Ground \nagainst Himachal Pradesh. His first half-century in domestic cricket happened at \nFeroze Shah Kotla, where he scored 70 runs against Haryana.[12] By the end of the \nseason, he had amassed a total of 172 runs, emerging as the highest run-scorer for \nhis side with an average of 34.40.[13] During the 2003–04 season of Polly Umrigar"),
  Document(metadata={'page': 0, 'type': 'text'}, page_content="his side with an average of 34.40.[13] During the 2003–04 season of Polly Umrigar \nTrophy, Kohli was appointed the captain of the U-15 team.[13] He scored 54 runs in \nDelhi's victory over Himachal Pradesh. In the next fixture against Jammu and \nKashmir, Kohli scored his maiden century with a score of 119. By the end of the \nseason, he had a total of 390 runs at an average of 78, which included two \ncenturies.[14] Towards the end

In [10]:


# Create custom FAISS index since we have precomputed embeddings
embeddings_array = np.vstack(all_embeddings)  
vector_store = FAISS.from_embeddings(
    text_embeddings=[(doc.page_content, emb) for doc, emb in zip(all_docs, embeddings_array)],
    embedding=None,  # We're using precomputed embeddings
    metadatas=[doc.metadata for doc in all_docs]
)
vector_store

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [11]:

# 🆕 NEW: Ollama Client Class
class OllamaLLaVA:
    def __init__(self, model="llava:7b", base_url="http://localhost:11434"):
        self.model = model
        self.base_url = base_url
        
    def generate_with_vision(self, prompt, images=None):
        """Generate response using Ollama's LLaVA model with vision capabilities."""
        url = f"{self.base_url}/api/generate"
        
        payload = {
            "model": self.model,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.1,
                "top_p": 0.9,
                "num_predict": 1000
            }
        }
        
        # Add images if provided (base64 format)
        if images:
            payload["images"] = images
        
        try:
            print(f"🤖 Querying {self.model}...")
            response = requests.post(url, json=payload, timeout=180)
            response.raise_for_status()
            
            result = response.json()
            return result.get("response", "No response generated")
            
        except requests.exceptions.RequestException as e:
            return f"❌ Ollama Error: {e}"
        except Exception as e:
            return f"❌ Unexpected Error: {e}"


In [12]:
# Initialize GPT-4 Vision model
llm = OllamaLLaVA(model="llava:7b")

In [13]:
def retrieve_multimodal(query, k=5):
    """Unified retrieval using CLIP embeddings for both text and images."""
    # Embed query using CLIP
    query_embedding = embed_text(query)
    
    # Search in unified vector store
    results = vector_store.similarity_search_by_vector(
        embedding=query_embedding,
        k=k
    )
    
    return results

In [14]:

# 🆕 NEW: Updated message creation for Ollama LLaVA
def create_ollama_multimodal_message(query, retrieved_docs):
    """Create a message with both text and images for Ollama LLaVA."""
    
    # Separate text and image documents
    text_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "text"]
    image_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "image"]
    
    # Build text context
    context_parts = []
    
    # Add text context
    if text_docs:
        text_context = "\n\n".join([
            f"📄 [Page {doc.metadata['page']}]: {doc.page_content}"
            for doc in text_docs
        ])
        context_parts.append(f"📝 **TEXT CONTENT:**\n{text_context}")
    
    # Collect images for Ollama
    images_to_analyze = []
    image_descriptions = []
    
    for doc in image_docs:
        image_id = doc.metadata.get("image_id")
        page = doc.metadata.get('page', '?')
        
        if image_id and image_id in image_data_store:
            # Add base64 image for Ollama
            images_to_analyze.append(image_data_store[image_id])
            image_descriptions.append(f"🖼️ Image from page {page} (analyzing below)")
    
    if image_descriptions:
        context_parts.append(f"📊 **VISUAL CONTENT:**\n" + "\n".join(image_descriptions))
    
    # Create comprehensive prompt for LLaVA
    full_context = "\n\n".join(context_parts)
    
    prompt = f"""🔍 **MULTIMODAL PDF ANALYSIS**

❓ **QUESTION:** {query}

📚 **DOCUMENT CONTEXT:**
{full_context}

🎯 **INSTRUCTIONS:**
1. Carefully read and understand the text content from the PDF
2. Analyze any images/charts/diagrams provided below
3. Provide a comprehensive answer combining insights from BOTH text and visual elements
4. Reference specific page numbers when mentioning information
5. If you see charts, graphs, or tables in images, describe what data they show
6. Be specific and detailed in your analysis

💡 **Your comprehensive analysis:**"""
    
    return {
        "prompt": prompt,
        "images": images_to_analyze
    }

In [15]:

# 🔄 UPDATED: Main pipeline function for Ollama
def multimodal_pdf_rag_pipeline(query):
    """Main pipeline for multimodal RAG using Ollama LLaVA."""
    
    # Retrieve relevant documents (SAME AS BEFORE ✅)
    context_docs = retrieve_multimodal(query, k=5)
    
    # Create Ollama-compatible message (NEW!)
    message_data = create_ollama_multimodal_message(query, context_docs)
    
    # Get response from Ollama LLaVA (NEW!)
    response = llm.generate_with_vision(
        prompt=message_data["prompt"],
        images=message_data["images"]
    )
    
    # Print retrieved context info (SAME AS BEFORE ✅)
    print(f"\n📋 Retrieved {len(context_docs)} documents:")
    for doc in context_docs:
        doc_type = doc.metadata.get("type", "unknown")
        page = doc.metadata.get("page", "?")
        if doc_type == "text":
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"  - 📝 Text from page {page}: {preview}")
        else:
            print(f"  - 🖼️ Image from page {page}")
    print("\n")
    
    return response

In [17]:

# 🧪 Testing with enhanced queries
if __name__ == "__main__":
    # 🔧 First, check if Ollama is running
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=5)
        models = [m["name"] for m in response.json()["models"]]
        print(f"✅ Ollama is running! Available models: {models}")
        
        if "llava:7b" not in models:
            print("❌ LLaVA:7b not found! Run: ollama pull llava:7b")
            exit(1)
        else:
            print("🎉 LLaVA:7b is ready!")
            
    except Exception as e:
        print(f"❌ Ollama not running! Start with: ollama serve\nError: {e}")
        exit(1)
    
    # Example queries optimized for vision capabilities
    queries = [
        "what does this pdf tells us about ?",
        "who is this person?",
        "describe the last paragraph on 2 lines "
    ]
    
    for query in queries:
        print(f"\n🔥 **Query:** {query}")
        print("-" * 50)
        answer = multimodal_pdf_rag_pipeline(query)
        print(f"**Answer:** {answer}")
        print("=" * 70)

✅ Ollama is running! Available models: ['llava:7b']
🎉 LLaVA:7b is ready!

🔥 **Query:** what does this pdf tells us about ?
--------------------------------------------------
🤖 Querying llava:7b...

📋 Retrieved 5 documents:
  - 📝 Text from page 1: praised Kohli for his performance, acknowledging its pivotal role in the team's 
success.[20] 
Kohli...
  - 📝 Text from page 0: trailing with a score of 130/5, with Kohli remaining unbeaten on 40 at the end of the 
day's play. T...
  - 📝 Text from page 1: team travelled to Malaysia for a tri-series against England Under-19s and Sri Lanka 
Under-19s, wher...
  - 📝 Text from page 0: his side with an average of 34.40.[13] During the 2003–04 season of Polly Umrigar 
Trophy, Kohli was...
  - 📝 Text from page 1: 2nd match, he contributed 83 runs to India's victory by 240 runs and an innings. He 
concluded the t...


**Answer:**  The PDF provides a detailed account of Virat Kohli's early cricket career, highlighting his impressive performances and de